In [2]:
import importlib

%load_ext autoreload
%autoreload 2

In [3]:
%load_ext dotenv
%dotenv

In [4]:
import os
from dotenv import load_dotenv

load_dotenv() 
ROOT = os.getenv("ROOT")
DATA = os.getenv("DATA")
SCRIPTS = os.getenv("SCRIPTS")

# Make root
os.chdir(SCRIPTS)
print(os.getcwd())

/home/callebalik/clinical_NLP_SE/scripts


In [5]:


from pathlib import Path
import pandas as pd
import numpy as np

from metrics.conll2003_utills import create_total_target_vector, conll2003_to_df, get_cleaned_label, create_total_target_df
from metrics.confusion_matrix import generate_confusion_matrix

cur = create_total_target_df(Path(f"{DATA}/interim/corpus/cur/"))

# [get_cleaned_label(label) for label in df["NE"]]
# dict = {"a1":a1, "a2":a2}

df = pd.DataFrame()
c = ["TOKEN", "NE"]
df["TOKEN"] = cur["TOKEN"]

df.insert(1, "NE", cur["NE"])

# Get all that does not contain DIS labels
df_sfno = (df[~df.NE.str.contains("DIS")]) 

df_sfno["CUR"] = [get_cleaned_label(label) for label in df_sfno["NE"]]
cur

/tmp/ipykernel_13309/3495451754.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sfno["CUR"] = [get_cleaned_label(label) for label in df_sfno["NE"]]


TOKEN POS CHUNK     NE
4 0    Kontaktorsak   _     O      O
  1               :   _     O      O
  2    Intoxikation   _     O  I-DIS
  3        Tidigare   _     O      O
  4       sjukdomar   _     O      O
...             ...  ..   ...    ...
2 319       Diagnos   _     O      O
  320             :   _     O      O
  321      Pneumoni   _     O  I-DIS
  322             ,   _     O      O
  323     misstänkt   _     O      O

[2338 rows x 4 columns]

In [11]:
values = ['SYM/FND', "NEG", "O"]

conditions = [
    (df_sfno.CUR.str.contains("FND")) | (df_sfno.CUR.str.contains("SYM")),
    (df_sfno.CUR.str.contains("NEG")),
    (df_sfno.CUR.str.contains("O"))
     ]

df_sfno['rmdis'] = np.select(conditions, values)
df_sfno[df_sfno.rmdis.str.contains("SYM")]


/tmp/ipykernel_13309/2153832643.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sfno['rmdis'] = np.select(conditions, values)


TOKEN     NE  CUR    rmdis
4 47               kräkts  I-FND  FND  SYM/FND
  51       svårkontaktbar  B-FND  FND  SYM/FND
  77               somnar  I-FND  FND  SYM/FND
  78               snabbt  I-FND  FND  SYM/FND
  79                undan  I-FND  FND  SYM/FND
...                   ...    ...  ...      ...
2 245         traumatiska  B-FND  FND  SYM/FND
  246              skador  I-FND  FND  SYM/FND
  252              febril  B-FND  FND  SYM/FND
  267              trauma  B-FND  FND  SYM/FND
  269  medvetandepåverkan  B-FND  FND  SYM/FND

[207 rows x 4 columns]

In [8]:
import app
import importlib
importlib.reload(app)

nlp = app.main()
type(nlp)

%load_ext autoreload
%autoreload 2

----Printing patterns, should be empty at this point----
[]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
nlp.get_pipe("entity_ruler").patterns[90:120]

[{'label': 'SYMFND', 'pattern': [{'LOWER': 'råttbettsfeber'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'erysipeloid'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'leptospiros'}]},
 {'label': 'SYMFND',
  'pattern': [{'LOWER': 'andra djurburna bakteriesjukdomar som ej klassificeras på annan plats'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'lepra'}]},
 {'label': 'SYMFND',
  'pattern': [{'LOWER': 'sjukdomar orsakade av andra mykobakterier'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'listerios'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'stelkramp hos nyfödd'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'obstetrisk stelkramp'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'annan stelkramp'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'difteri'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'kikhosta'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'scharlakansfeber'}]},
 {'label': 'SYMFND', 'pattern': [{'LOWER': 'meningokockinfektion'}]},
 {'label': 'SYMFND',
  'patter

In [15]:
df_sfno.TOKEN 

4  0      Kontaktorsak
   1                 :
   3          Tidigare
   4         sjukdomar
   5                 :
              ...     
2  318               .
   319         Diagnos
   320               :
   322               ,
   323       misstänkt
Name: TOKEN, Length: 2297, dtype: object

In [28]:
    
doc = []
[doc.append(token) for token in df_sfno.TOKEN]

ds = " ".join(doc)
ndoc = nlp(ds)

In [54]:
dfn = pd.DataFrame()

ent = []
for t in ndoc:
    # print(type(t))
    print(t.ent_type_)
    if t.ent_type_  != "":
        ent.append(t.ent_type_)
    else:
        s = "O"
        ent.append(s)
dfn["NLP"] = [t for t in ndoc]
ent










NEG




























NEG
























NEG
























































NEG
SYMFND

NEG



















































NEG









NEG





NEG



















NEG










NEG


NEG





NEG















SYMFND



NEG






NEG


LOC






LOC


PRS












TME
TME
TME
TME
























NEG




















PRS


































TME
TME








































TME


TME



NEG




TME
TME


















NEG


























NEG







TME
TME







































NEG













NEG




















TME
















TME
TME
TME









TME
TME
























SYMFND















SYMFND













ORG











































NEG








MSR
MSR






NEG























NEG




































NEG



















NEG





NEG
NEG











MSR
MSR


TME
TME
TME


['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'NEG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'NEG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'NEG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'NEG',
 'SYMFND',
 'O',
 'NEG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv() 
ROOT = os.getenv("ROOT")
DATA = os.getenv("DATA")
SCRIPTS = os.getenv("SCRIPTS")

# Make root
os.chdir(SCRIPTS)
print(os.getcwd())


def getdoc():
    open(f"{DATA}/raw/corpus/scr/)

In [8]:

from spacy.training import Corpus
from pathlib import Path

corpus = Corpus(Path(f"{DATA}/processed/corpus/cur/"), limit=20, gold_preproc=True)
examples= corpus(nlp)

a = []
for k in examples:
    # print(type(k))
    # print(k.predicted)
    a.append(k)


i = 6
a[i].text
a[i].predicted
a[i].get_aligned_ner()

for b in a[i].predicted:
    print(f"{b.ent_type_} : {b.text}")
print(a[i].reference[1].ent_type_)
# [print(f"item #{index + 1} = {items[index]}") for index, item in enumerate(items)]

docs = [eg.predicted for eg in examples]
# predictions, _ = model.begin_update(docs)
# set_annotations(docs, predictions)
gold_labels = []
for i, eg in enumerate(examples):
    # for j, label in enumerate(all_labels):
        # gold_labels[i][j] = eg.reference.cats.get(label, 0.0)
    gold_labels[i] = eg.reference.cats.get(label, 0.0)
    
print(gold_labels)

 : Ej
 : noterat
 : feber
 : ,
 : men
 : ej
 : eller
 : mätt
 : tempen
 : .

[]


In [ ]:
from spacy.training import Alignment
from spacy.tokens import Doc
from spacy.training import Example

def get_all_ner_predictions(text):
    doc = nlp(text)
    entities = [(e.start_char, e.end_char, e.label_) for e in doc.ents]
    bilou_entities = offsets_to_biluo_tags(doc, entities)
    return bilou_entities


bert_tokens = ["obama", "'", "s", "podcast"]
spacy_tokens = ["obama", "'s", "podcast"]
alignment = Alignment.from_strings(bert_tokens, spacy_tokens)
a2b = alignment.x2y
assert list(a2b.dataXd) == [0, 1, 1, 2]

In [80]:

words = ["Mrs", "Smith", "flew", "to", "New York"]
doc = Doc(nlp.vocab, words=words)
entities = [(0, 9, "PERSON"), (18, 26, "LOC")]
gold_words = ["Mrs Smith", "flew", "to", "New", "York"]
example = Example.from_dict(doc, {"words": gold_words, "entities": entities})
ner_tags = example.get_aligned_ner()
assert ner_tags == ["B-PERSON", "L-PERSON", "O", "O", "U-LOC"]
ner_tags

['B-PERSON', 'L-PERSON', 'O', 'O', 'U-LOC']

In [15]:
from spacy.tokens import DocBin

DOCBIN_PATH = Path(f"{DATA}/processed/corpus/cur")
doc_bin = DocBin().from_disk(f"{DOCBIN_PATH}/1.spacy")
doc_bin.merge(DocBin().from_disk(f"{DOCBIN_PATH}/2.spacy"))



docs = list(doc _bin.get_docs(nlp.vocab))
# d = doc_bin.get_docs(nlp.vocab)

docs[0]

Kontaktorsak : Dyspné Tidigare sjukdomar : Insulin och tablettbehandlad DM2 . Tablettbehandlad hypertoni . Höga blodfetter . Socialt : Ensamboende , hemtjänst * 1 . Aktuellt : Insjuknat för tre dagar sedan med andnöd vid gång . Tillkomst av hosta under gårdagen . Ej noterat feber , men ej eller mätt tempen . Idag nedsatt AT , svårt att gå . Tagit sina mediciner som vanligt . Sökt VC som hänvisat hit pga destruering . Inte haft några utslag . Ej kontakt med någon annan som varit sjuk . Ej varit ute och rest . Ingen smärta vid miktion , buksmärta eller ÖNH symptom . Idag sämre , kan ej mobilisera sig i hemmet . Inkommer på remiss . Inga kräkningar eller GI symptom . Status : AT : Gott , opåverkad i vila , ingen samtalsdyspné . Cor : Normofrekvent regelbunden rytm , inga hörbara bi eller blåsljud . Pulm : I stort vesikulära antingsljud bilat , dämpat basalt höger . Inget förlängt expirium . Buk : Överallt muk och oöm . Ingen lokal peritonit . Lokalstatus : Underben smala och gracila , ing

In [50]:
d

<generator object DocBin.get_docs at 0x7fd63f652b30>

In [45]:

from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
from spacy.tokens import Doc
from spacy.training import Example

from spacy.tokens import Doc
from spacy.training import Example

words = docs[0].text
predicted = Doc(nlp.vocab, words=words)
reference = docs[0]
example = Example(predicted, reference)

# for doc in docs:
#     ents = list(doc.ents)
    # for e in ents:
    #     print(f"{e.text} | {e.label_}")


# np_array = doc.to_array([LOWER, POS, ENT_TYPE, IS_ALPHA])
# np_array


In [41]:
example.get_aligned_ner()

ValueError: [E949] Unable to align tokens for the predicted and reference docs. It is only possible to align the docs when both texts are the same except for whitespace and capitalization. The predicted tokens start with: ['hello', 'world', '!']. The reference tokens start with: ['Kontaktorsak', ':', 'Medicinlarm', ',', 'dyspné', 'Tidigare', 'sjukdomar', ':', 'Waranbehandlat', 'FF'].

In [ ]:
from plot_confusion_matrix import plot_confusion_matrix

from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib as mpl
import matplotlib.gridspec as gridspec


top = cm.get_cmap('Oranges_r', 128)
bottom = cm.get_cmap('Blues', 128)

newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
newcmp = ListedColormap(newcolors, name='OrangeBlue')

viridisBig = cm.get_cmap('viridis_r', 512)
viridis_new = ListedColormap(viridisBig(np.linspace(0.15, 1, 256)))

viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(0, 1, 256))
pink = np.array([248/256, 24/256, 148/256, 1])
newcolors[:5, :] = pink
newcmp = ListedColormap(newcolors)

coolw = cm.get_cmap('coolwarm', 512)
coolw_new = ListedColormap(coolw(np.linspace(0.15, 1, 256)))

color = coolw_new
color = plt.cm.YlGnBu
# color = plt.cm.coo


classes1 = ["SYM", "FND","NEG","O"]

classes2 = ["SYM/FND","NEG","O"]

font = {#'family': 'serif',
        # 'color':  'black',
        #'weight': 'normal',
        'size': 10,
        }

px = 1/plt.rcParams['figure.dpi']  # pixel in inches
cm = 1/2.54  # centimeters in inches
# Providing the axs
# fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(40*cm, 10*cm), constrained_layout=True)
# # First subplt
# cm1, ax1, im1 = plot_confusion_matrix(df_sfno.A1, df_sfno.A2, classes1, ax=axs[0], normalize=True)
# axs[0].figure.colorbar(im1, ax=axs[0], fraction=0.05)
# # Second subplt
# cm2, ax2, im2 = plot_confusion_matrix(df_sfno.A1_sf, df_sfno.A2_sf, classes2, ax=axs[1], normalize=True)
# axs[1].figure.colorbar(im2, ax=axs[1], fraction=0.15)



fig2 = plt.figure(tight_layout=True)
spec2 = gridspec.GridSpec(1,1)

ax = fig2.add_subplot(spec2[0])

cm1, ax1, im1 = plot_confusion_matrix(df_sfno.A1, df_sfno.A2, classes1, ax=ax, normalize=True, cmap=coolw_new)
axs[0].figure.colorbar(im1, ax=axs[0], fraction=0.05)
# Second subplt

ax.set(title=None)
ax.set_xlabel(xlabel='annotator2', fontdict=font)
ax.set_ylabel(ylabel='annotator1', fontdict=font)

axs = fig2.get_axes()
texts = ['(A)', '(B)']
for a,l in zip(axs, texts):

    # X = a.get_position().x0
    # Y = a.get_position().y1    
    # fig2.text(X - .1, Y + .018, l, size=10, weight='bold')
    a.annotate(l, xy=(-0.45, 1.1), xycoords="axes fraction", fontsize=11, weight = 'bold')
# plt.subplots_adjust(left=-0.45)

# plt.tight_layout() #This to avoid overlap of labels and titles across plots

plt.show()

plt.savefig(f"{DATA}/processed/IAA_SYM_FND", )

In [ ]:
output